In [1]:
import numpy as np
import pandas as pd


In [2]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv("iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


In [3]:
def calculate_leaf_value(Y):
    Y = list(Y)
    return max(Y, key=Y.count)

In [4]:
def gini_index(y):
    class_labels = np.unique(y)
    gini = 0
    for cls in class_labels:
        p_cls = len(y[y == cls]) / len(y)
        gini += p_cls**2
    return 1 - gini

In [5]:
def entropy(y):
    class_labels = np.unique(y)
    entropy = 0
    for cls in class_labels:
        p_cls = len(y[y == cls]) / len(y)
        entropy += -p_cls * np.log2(p_cls)
    return entropy

In [6]:
def information_gain(parent, l_child, r_child, mode="entropy"):
    weight_l = len(l_child) / len(parent)
    weight_r = len(r_child) / len(parent)
    if mode == "gini":
        gain = gini_index(parent) - (weight_l * gini_index(l_child) + weight_r * gini_index(r_child))
    else:
        gain = entropy(parent) - (weight_l * entropy(l_child) + weight_r * entropy(r_child))
    return gain

In [7]:
def split(dataset, feature_index, threshold):
    dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
    dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
    return dataset_left, dataset_right

In [8]:
def get_best_split(dataset, num_samples, num_features, min_samples_split, max_depth):
    best_split = {}
    max_info_gain = -float("inf")
    for feature_index in range(num_features):
        feature_values = dataset[:, feature_index]
        possible_thresholds = np.unique(feature_values)
        for threshold in possible_thresholds:
            dataset_left, dataset_right = split(dataset, feature_index, threshold)
            if len(dataset_left) > 0 and len(dataset_right) > 0:
                y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                curr_info_gain = information_gain(y, left_y, right_y, "gini")
                if curr_info_gain > max_info_gain:
                    best_split["feature_index"] = feature_index
                    best_split["threshold"] = threshold
                    best_split["dataset_left"] = dataset_left
                    best_split["dataset_right"] = dataset_right
                    best_split["info_gain"] = curr_info_gain
                    max_info_gain = curr_info_gain
    return best_split

In [9]:
def build_tree(dataset, min_samples_split, max_depth, curr_depth=0):
    X, Y = dataset[:, :-1], dataset[:, -1]
    num_samples, num_features = np.shape(X)
    if num_samples >= min_samples_split and curr_depth <= max_depth:
        best_split = get_best_split(dataset, num_samples, num_features, min_samples_split, max_depth)
        if best_split["info_gain"] > 0:
            left_subtree = build_tree(best_split["dataset_left"], min_samples_split, max_depth, curr_depth + 1)
            right_subtree = build_tree(best_split["dataset_right"], min_samples_split, max_depth, curr_depth + 1)
            return {"feature_index": best_split["feature_index"], 
                    "threshold": best_split["threshold"], 
                    "left": left_subtree, 
                    "right": right_subtree, 
                    "info_gain": best_split["info_gain"]}
    leaf_value = calculate_leaf_value(Y)
    return {"value": leaf_value}

In [10]:
def print_tree(tree=None, indent=" "):
    if tree is None:
        return
    if "value" in tree:
        print(tree["value"])
    else:
        print("X_" + str(tree["feature_index"]), "<=", tree["threshold"], "?", tree["info_gain"])
        print("%sleft:" % (indent), end="")
        print_tree(tree["left"], indent + indent)
        print("%sright:" % (indent), end="")
        print_tree(tree["right"], indent + indent)

In [11]:
def fit(X, Y, min_samples_split=2, max_depth=2):
    dataset = np.concatenate((X, Y), axis=1)
    return build_tree(dataset, min_samples_split, max_depth)

In [12]:
def make_prediction(x, tree):
    if "value" in tree:
        return tree["value"]
    feature_val = x[tree["feature_index"]]
    if feature_val <= tree["threshold"]:
        return make_prediction(x, tree["left"])
    else:
        return make_prediction(x, tree["right"])

In [13]:
def predict(X, tree):
    return [make_prediction(x, tree) for x in X]

In [14]:
def accuracy_score(y_true, y_pred):
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    return correct / len(y_true)

In [15]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)
tree = fit(X_train, Y_train, min_samples_split=3, max_depth=3)
print_tree(tree)
Y_pred = predict(X_test, tree)
print(accuracy_score(Y_test, Y_pred))

X_2 <= 1.9 ? 0.33741385372714494
 left:Iris-setosa
 right:X_3 <= 1.5 ? 0.427106638180289
  left:X_2 <= 4.9 ? 0.05124653739612173
    left:Iris-versicolor
    right:Iris-virginica
  right:X_2 <= 5.0 ? 0.019631171921475288
    left:X_1 <= 2.8 ? 0.20833333333333334
        left:Iris-virginica
        right:Iris-versicolor
    right:Iris-virginica
0.9333333333333333


In [19]:
#001